Detección de caras con webcam

In [61]:
import cv2
import time
import FaceNormalizationUtils as faceutils
# My face detectors interface
import FaceDetectors
import math
import numpy as np

def rotate_from_origin(xy, radians):
    """Only rotate a point around the origin (0, 0)."""
    x, y = xy
    xx = x * math.cos(radians) + y * math.sin(radians)
    yy = -x * math.sin(radians) + y * math.cos(radians)

    return xx, yy

def cv2_grad(frame, pt1, pt2, color_start, color_end, width, divs):
    pt_from = pt1
    pt_to = None
    color_inter = None
    for i in range(divs):
        frac_pt = (i+1)/divs
        frac_color = i/(divs-1)

        color_inter = [int(ch2*frac_color + ch1*(1-frac_color)) for ch1, ch2 in zip(color_start, color_end)]
        pt_to = [int(coord2*frac_pt + coord1*(1-frac_pt)) for coord1, coord2 in zip(pt1, pt2)]
        
        cv2.line(frame, pt_to, pt_from, color_inter, width)
        pt_from = pt_to

En el bucle de procesamiento, las teclas 'd' y 'e' `permiten respectivamenet cambiar de modelo de detección de caras, y en su caso de máscara de detección del rostro.

La ejecución de la siguiente celda produce error al no disponer de los archivos shape_predictor_5_face_landmarks.dat y shape_predictor_68_face_landmarks.dat que por su tamaño no se incluyeron en el repositorio. Pueden descargarse desde el enlace proporcionado en el campus virtual (opción aconsejada), o
desde el [repositorio de archivos de dlib](http://dlib.net/files/).



In [68]:
normalizatorHS = faceutils.Normalization()

# Face detectors interface
FDet = FaceDetectors.FaceDetector()

faceD = FDet.FaceDetectors[1]
eyesD = FDet.EyeDetectors[1]

size = 4
historical = [[0 for j in range(size)] for i in range(3)]
pointer = 0

def put_xyz_ang(x_ang, y_ang, z_ang):
    global historical
    global pointer
    global size
    thresh = 0.35
        
    historical[0][pointer] = x_ang if abs(y_ang) < thresh else historical[0][(pointer+size-1)%size]
    historical[1][pointer] = y_ang
    historical[2][pointer] = z_ang
    pointer += 1
    pointer %= size

def get_xyz_ang():
    global historical
    return [sum(historical[i])/len(historical[i]) for i in range(3)]

debug = True
showLaser = True

laser_diameter = 10
laser_length = 1000
laser_color = (0, 0, 255)
compensation_x_ang = 5

# Fonts
font = cv2.FONT_HERSHEY_SIMPLEX

# Webcam connection
cap = cv2.VideoCapture(0)
# Check for other cameras
if not cap.isOpened():
    cap = cv2.VideoCapture(1)
    if not cap.isOpened():
        cap = cv2.VideoCapture(0)
        if not cap.isOpened():
            print('Camera error')
            exit(0)
        else:
            print('Camera 0')
    else:
        print('Camera 1')
else:
    print('Camera 0')


#Set camera resolution
disp_w = 640
disp_h = 480
cap.set(3,disp_w)
cap.set(4,disp_h)
while True:
    # Get frame
    ret, frame = cap.read()

    # Search face with a specific setup for face and eye detection
    values = FDet.SingleFaceEyesDetection(frame, faceD, eyesD)
    if values is not None:
        face, eyes, shape = values

        #draws face container
        [x, y , w, h] = face
        if x > -1:

            # draws eyes and mask if available
            [lex, ley, rex, rey] = eyes
            if lex > -1:
                # Ángulo positivo = girar en sentido antihorario
                # x = mirar abajo, y = mirar izquierda, z = girar derecha
                # Unidad = radianes
                z_ang = math.atan((ley - rey) / (lex - rex))

                ab = (shape[14]-shape[2])
                ap = (shape[33]-shape[2])
                distCheeks = np.linalg.norm(ab)
                distNoseProjected = np.linalg.norm(np.dot(ap, ab) / np.dot(ab, ab) * ab)
                y_ang = (distNoseProjected / distCheeks - 0.5) * 0.6
                
                x_ang = rotate_from_origin((lex, ley), z_ang)[1]-rotate_from_origin(shape[0], z_ang)[1] \
                if y_ang > 0 else rotate_from_origin((rex, rey), z_ang)[1]-rotate_from_origin(shape[16], z_ang)[1] - compensation_x_ang
                x_ang /=  120

                open_mouth = 1 + math.hypot(shape[62,0]-shape[66,0], shape[62,1]-shape[66,1])/10

                put_xyz_ang(x_ang, y_ang, z_ang) # Poner en el bufer
                x2, y2, z2 = get_xyz_ang() # Sacar la media

                if debug: # Puntos dlib68 + linea con el ángulo Z
                    for (x, y) in shape:
                        cv2.circle(frame, (x, y), 2, (255, 255, 255), -1)

                    for i in (33, 2, 14):
                        cv2.circle(frame, (shape[i,0], shape[i,1]), 2, (255, 0, 0), -1)

                    if y_ang > 0:
                        cv2.circle(frame, ((int)(lex), (int)(ley)), 4, (0, 0, 255), -1)
                    else:
                        cv2.circle(frame, ((int)(rex), (int)(rey)), 4, (0, 255, 0), -1)
                    pt = rotate_from_origin((0, 100), z2)
                    #cv2.line(frame, (int(disp_w/2), int(disp_h/2)), (int(disp_w/2+pt[0]), int(disp_h/2-pt[1])), (0, 0, 0), laser_diameter)

                if showLaser:
                    looking_pt = rotate_from_origin((math.sin(y2)*laser_length, math.sin(-x2)*laser_length), z2)
                    looking_pt_l = (int(looking_pt[0]+lex), int(ley-looking_pt[1]))
                    looking_pt_r = (int(looking_pt[0]+rex), int(rey-looking_pt[1]))

                    if looking_pt[0] < 0: # Cambiar el orden de las llamadas
                        cv2_grad(frame, (int(lex), int(ley)), looking_pt_l, (255, 255, 255), laser_color, int(laser_diameter*open_mouth), 15)
                        cv2_grad(frame, (int(rex), int(rey)), looking_pt_r, (255, 255, 255), laser_color, int(laser_diameter*open_mouth), 15)
                    else:
                        cv2_grad(frame, (int(rex), int(rey)), looking_pt_r, (255, 255, 255), laser_color, int(laser_diameter*open_mouth), 15)
                        cv2_grad(frame, (int(lex), int(ley)), looking_pt_l, (255, 255, 255), laser_color, int(laser_diameter*open_mouth), 15)

    # Show resulting image
    cv2.putText(frame, faceD, (10, 20), font, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
    cv2.putText(frame, eyesD, (50, 20), font, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
    cv2.imshow('Cam', frame)
    
    # Esc to finish
    tec = cv2.waitKey(40)
    if tec & tec == 27:  # Esc
        break

# Close windoews and release camera
cap.release()
cv2.destroyAllWindows()

Camera 0
